In [1]:
import datetime
import json
import os
import firebase_admin
from firebase_admin import credentials, firestore
from googletrans import Translator
from faker import Faker

ModuleNotFoundError: No module named 'faker'

In [ ]:
f = open("esl_words.txt")
esl_words = [line.rstrip('\n') for line in f]
f.close()

In [ ]:
esl_words = set(esl_words)
print(len(esl_words))

In [ ]:
if "tightly" in esl_words: print("hello")

In [ ]:
cred = credentials.Certificate(os.getcwd() + "/cogs121-goduck-firebase-adminsdk-f5baa-73bc1269cd.json")
firebase_admin.initialize_app(cred)

In [ ]:
translator = Translator()
fake = Faker()

In [ ]:
db = firestore.client()
news_content_ref = db.collection('news_content')
news_overview_ref = db.collection('news_overview')

In [ ]:
with open(os.getcwd() + "/fake_news.json", 'r') as f:
    data = temp = json.load(f)

In [ ]:
def processNews(title, source, text, category):
    news_content_obj = {"title": title}
    news_content_obj["source"] = source
    news_content_obj["category"] = category
    news_content_obj["translated_title_zh"] = translator.translate(title, dest='zh-cn', src='en').text
    news_content_obj["translated_title_es"] = translator.translate(title, dest='es', src='en').text
    news_content_obj["translated_title_hi"] = translator.translate(title, dest='hi', src='en').text

    p_list = text.split("\n")
    news_content_obj["text"] = []
    
    for p in p_list:
        temp = {}
        array_p = p.split(" ")
        temp_arr = []
        for w in array_p:
            if w.lower() in esl_words:
                t1 = translator.translate(w, dest='zh-cn', src='en').text
                t2 = translator.translate(w, dest='es', src='en').text
                t3 = translator.translate(w, dest='hi', src='en').text  
                w = '<esl data="' + t1 + ',' + t2 + ',' + t3 + '">' + w + "</esl>"
            temp_arr.append(w)
        temp['zh'] = translator.translate(p, dest='zh-cn', src='en').text
        temp['es'] = translator.translate(p, dest='es', src='en').text
        temp['hi'] = translator.translate(p, dest='hi', src='en').text
        temp['en'] = " ".join(temp_arr)
        news_content_obj["text"].append(temp)
        
    news_content_obj['publish_time'] = fake.past_datetime(start_date="-3d", tzinfo=None)
    
    news_content_obj_ref = news_content_ref.add(news_content_obj)
    news_content_obj_id = news_content_obj_ref[1].get().id
    
    news_overview_obj = {"title": title}
    news_overview_obj['content_id'] = news_content_obj_id
    news_overview_obj["source"] = source 
    news_overview_obj["publish_time"] = news_content_obj['publish_time']
    news_overview_obj["translated_title_zh"] = news_content_obj["translated_title_zh"]
    news_overview_obj["translated_title_es"] = news_content_obj["translated_title_es"]
    news_overview_obj["translated_title_hi"] = news_content_obj["translated_title_hi"]
    news_overview_obj["category"] = news_content_obj["category"]
    
    news_overview_ref.add(news_overview_obj)

In [ ]:
#for n in data["news"]['business']:
n = data['news']['business'][1]
processNews(n['title'], n['source'], n['text'], 1)

In [ ]:
for n in data["news"]['tech']:
    processNews(n['title'], n['source'], n['text'], 0)

In [ ]:
for n in data["news"]['business']:
    processNews(n['title'], n['source'], n['text'], 1)